# 🌳 Árvore de Decisão — Churn do Produto A
**Pós-Graduação BI & Analytics · Análise Complementar ao Logit**

---

### Por que Árvore de Decisão aqui?

O Logit entregou AUC ~0.83 e interpretação via coeficientes lineares.  
A árvore de decisão vai mostrar **como o modelo raciocina em regras** — a lógica em forma de `if/else`.  
Isso tem duas vantagens práticas:

- **Interpretabilidade visual direta** — qualquer pessoa de negócio consegue seguir o fluxo
- **Captura interações** — o modelo descobre que `segmento_A = 1 E nps_score < 5` é especialmente perigoso, sem precisar que a gente especifique essa interação

### Relação com os outros modelos

| Modelo | Interpreta como | Captura interações | AUC típico |
|---|---|---|---|
| Logit | Coeficientes lineares | ❌ Não | 0.82–0.86 |
| **Árvore de Decisão** | **Regras if/else** | **✅ Sim** | 0.80–0.88 |
| XGBoost | SHAP values | ✅ Sim (muito) | 0.87–0.92 |

---
### Roteiro
1. Importar e preparar os dados (mesmo pré-processamento do Logit)
2. Problema de profundidade: overfitting vs. underfitting
3. Encontrar a profundidade ótima
4. Treinar o modelo final
5. Visualizar a árvore (interpretação visual)
6. Extrair regras de decisão em texto
7. Avaliar: Matriz de Confusão, AUC-ROC
8. Importância das features
9. Comparar com Logit
10. Saída acionável: segmentos de risco com regras claras

In [ ]:
# ── Imports ────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_auc_score, roc_curve,
    accuracy_score, f1_score,
    precision_score, recall_score
)

plt.rcParams.update({
    'figure.dpi': 130,
    'font.family': 'DejaVu Sans',
    'axes.spines.top': False,
    'axes.spines.right': False,
})

COR_CHURN  = '#c84b2f'
COR_OK     = '#2f6ec8'
COR_NEUTRO = '#94a3b8'

print('✅ Bibliotecas carregadas')

## 1. Preparar os Dados

In [ ]:
# ── Carregar dados ─────────────────────────────────────────────────
URL = 'https://raw.githubusercontent.com/SEU_USUARIO/ml-bi-analytics-aula/main/data/clientes_produto_A.csv'
# df = pd.read_csv('clientes_produto_A.csv')  # local
df = pd.read_csv(URL)
print(f'Shape: {df.shape}')

# ── Pré-processamento (idêntico ao notebook do Logit) ──────────────
df_model = pd.get_dummies(
    df.drop(columns=['id_cliente', 'segmento', 'prob_churn_real'], errors='ignore'),
    columns=['regiao', 'porte'],
    drop_first=True
)

TARGET   = 'reduziu_compras'
X        = df_model.drop(columns=[TARGET])
y        = df_model[TARGET]
FEATURES = list(X.columns)

# Imputar missing com mediana
imputer = SimpleImputer(strategy='median')
X_imp   = pd.DataFrame(imputer.fit_transform(X), columns=FEATURES)

# Split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X_imp, y, test_size=0.2, stratify=y, random_state=42
)

print(f'Treino: {X_train.shape} | Teste: {X_test.shape}')
print(f'Churn rate: {y.mean():.1%}')
print()
print('📌 Árvore de decisão NÃO precisa de StandardScaler.')
print('   O algoritmo usa thresholds — a escala das variáveis é irrelevante.')

## 2 + 3. O Problema da Profundidade

> **Profundidade pequena** → underfitting: árvore simplista, não captura padrões reais.  
> **Profundidade grande** → overfitting: decora o treino, generaliza mal.  
> Encontrar o ponto de equilíbrio é o principal hiperparâmetro da árvore.

In [ ]:
# ── Curva de complexidade: AUC treino vs. teste por profundidade ───
depths     = range(1, 14)
aucs_train = []
aucs_test  = []
aucs_cv    = []

for d in depths:
    dt_tmp = DecisionTreeClassifier(
        max_depth=d, random_state=42,
        class_weight='balanced', min_samples_leaf=15
    )
    dt_tmp.fit(X_train, y_train)
    aucs_train.append(roc_auc_score(y_train, dt_tmp.predict_proba(X_train)[:, 1]))
    aucs_test.append(roc_auc_score(y_test,  dt_tmp.predict_proba(X_test)[:, 1]))

    cv = cross_val_score(
        DecisionTreeClassifier(max_depth=d, random_state=42,
                               class_weight='balanced', min_samples_leaf=15),
        X_imp, y, cv=5, scoring='roc_auc'
    )
    aucs_cv.append(cv.mean())

best_depth = list(depths)[np.argmax(aucs_cv)]

fig, ax = plt.subplots(figsize=(10, 4.5))
ax.plot(depths, aucs_train, 'o-', color=COR_NEUTRO, lw=2, ms=6,
        label='AUC — Treino (otimista)')
ax.plot(depths, aucs_test,  's-', color=COR_OK,    lw=2, ms=6,
        label='AUC — Teste')
ax.plot(depths, aucs_cv,    '^-', color=COR_CHURN, lw=2, ms=6,
        label='AUC — Cross-val 5-fold (mais confiável)')
ax.axvline(best_depth, color='#1a1a1a', lw=1.5, linestyle='--',
           label=f'Profundidade ótima: {best_depth}')
ax.fill_betweenx([0.5, 1.01], best_depth - 0.4, best_depth + 0.4,
                 alpha=0.08, color='#1a1a1a')
ax.annotate('← Underfitting', xy=(2, aucs_cv[1]),
            xytext=(2.4, 0.72), fontsize=8.5, color='#888')
ax.annotate('Overfitting →\n(treino ↑, teste ↓)', xy=(10, aucs_train[9]),
            xytext=(9.2, 0.84), fontsize=8.5, color='#888', ha='center')
ax.set(xlabel='Profundidade máxima (max_depth)', ylabel='AUC-ROC',
       title='Curva de Complexidade — Underfitting vs. Overfitting',
       xlim=(0.8, max(depths) + 0.2), ylim=(0.65, 1.01))
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

print(f'Profundidade ótima (cross-val): {best_depth}')
print(f'AUC cross-val: {aucs_cv[best_depth-1]:.3f}')
print(f'AUC teste:     {aucs_test[best_depth-1]:.3f}')

## 4. Treinar o Modelo Final

In [ ]:
# ── Modelo final ───────────────────────────────────────────────────
dt = DecisionTreeClassifier(
    max_depth         = best_depth,
    class_weight      = 'balanced',
    min_samples_leaf  = 20,    # nó folha precisa de pelo menos 20 amostras
    min_samples_split = 40,    # nó interno precisa de pelo menos 40 para dividir
    random_state      = 42
)
dt.fit(X_train, y_train)

y_pred  = dt.predict(X_test)
y_proba = dt.predict_proba(X_test)[:, 1]

print('Parâmetros do modelo final:')
print(f'  max_depth:           {dt.max_depth}')
print(f'  Nós totais:          {dt.tree_.node_count}')
print(f'  Folhas (segmentos):  {dt.get_n_leaves()}')
print(f'  Features utilizadas: {(dt.feature_importances_ > 0).sum()} de {len(FEATURES)}')
print()
print('Métricas no conjunto de teste:')
print(f'  AUC-ROC  : {roc_auc_score(y_test, y_proba):.3f}')
print(f'  Acurácia : {accuracy_score(y_test, y_pred):.3f}')
print(f'  Recall   : {recall_score(y_test, y_pred):.3f}')
print(f'  Precisão : {precision_score(y_test, y_pred):.3f}')
print(f'  F1-Score : {f1_score(y_test, y_pred):.3f}')

## 5. Visualizar a Árvore

> Esta é a principal vantagem da árvore de decisão sobre outros modelos:  
> **qualquer pessoa de negócio consegue seguir a lógica de cima para baixo.**

In [ ]:
# ── Nomes legíveis para os nós ────────────────────────────────────
nomes_legiveis = {
    'segmento_A':            'Segmento A',
    'var_compras_6m':        'Var. Compras 6m (%)',
    'freq_compras_trim':     'Freq. Compras/Trim',
    'ticket_medio_ratio':    'Ticket Médio Ratio',
    'tempo_cliente_anos':    'Tempo Cliente (anos)',
    'nps_score':             'NPS Score',
    'canal_digital':         'Canal Digital',
    'vendedor_rotatividade': 'Rot. Vendedor',
    'inadimplencia_hist':    'Inadimplência Hist.',
}
feature_names_plot = [nomes_legiveis.get(f, f) for f in FEATURES]

# ── Plotar árvore ─────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(22, 9))
plot_tree(
    dt,
    feature_names = feature_names_plot,
    class_names   = ['Não reduziu', 'Reduziu'],
    filled        = True,
    rounded       = True,
    fontsize      = 8,
    ax            = ax,
    impurity      = False,
    precision     = 2,
)
ax.set_title(
    'Árvore de Decisão — Churn do Produto A\n'
    '(azul = majoritariamente não-churn  |  laranja = majoritariamente churn)',
    fontsize=12, fontweight='bold', pad=14
)
plt.tight_layout()
plt.show()

print('💡 Como ler a árvore:')
print('   • Cada nó mostra a regra de divisão (ex: NPS ≤ 4.5)')
print('   • Cor mais intensa = maior pureza do nó')
print('   • Folhas laranjas = segmento de alto risco de churn')
print('   • Folhas azuis   = segmento de baixo risco')

## 6. Regras de Decisão em Texto

> A árvore pode ser exportada como regras `if/else` em linguagem natural.  
> O time de negócio pode aplicar essas regras **sem abrir o Python**.

In [ ]:
# ── Exportar como texto ───────────────────────────────────────────
regras = export_text(
    dt,
    feature_names = feature_names_plot,
    decimals      = 2,
    show_weights  = True
)
print('=== REGRAS DE DECISÃO (formato if/else) ===')
print(regras)

In [ ]:
# ── Resumo das folhas: quais segmentos têm alto risco? ────────────
tree_ = dt.tree_

print('=== FOLHAS DA ÁRVORE ===\n')
print(f"{'Folha':>6}  {'N amostras':>10}  {'% churn':>9}  {'Classificação'}")
print('─' * 55)

for node_id in range(tree_.node_count):
    if tree_.children_left[node_id] != -1:
        continue  # não é folha
    n_total   = int(tree_.n_node_samples[node_id])
    n_class   = tree_.value[node_id][0]
    pct_churn = n_class[1] / n_class.sum()
    classe    = '🔴 ALTO RISCO' if pct_churn > 0.55 else '🟢 Baixo risco'
    print(f'  {node_id:>4}  {n_total:>10,}  {pct_churn:>8.1%}  {classe}')

print()
print('💡 Cada folha é um segmento homogêneo.')
print('   Folhas vermelhas → ação comercial prioritária.')

In [ ]:
# ── Em que nível cada feature aparece? ────────────────────────────
# Features no nível 0 (raiz) têm o maior poder discriminatório global.
feature_depth = {}

def traverse(node, depth):
    if tree_.children_left[node] == -1:
        return
    fname = feature_names_plot[tree_.feature[node]]
    feature_depth.setdefault(fname, []).append(depth)
    traverse(tree_.children_left[node],  depth + 1)
    traverse(tree_.children_right[node], depth + 1)

traverse(0, 0)

print('Features por nível da árvore (nível 0 = raiz = mais discriminante):\n')
for nivel in range(best_depth):
    feats = [f for f, ds in feature_depth.items() if nivel in ds]
    if feats:
        print(f'  Nível {nivel}: {" | ".join(feats)}')

## 7. Avaliação — Matriz de Confusão e AUC-ROC

In [ ]:
# ── Métricas completas ─────────────────────────────────────────────
cm  = confusion_matrix(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
fpr, tpr, _ = roc_curve(y_test, y_proba)

fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

# Matriz de confusão
axes[0].imshow(cm, cmap='Blues')
labels_cm = [['TN\n(acertou não-churn)', 'FP\n(alarme falso)'],
             ['FN\n(perdeu churn)',       'TP\n(pegou churn)']]
for i in range(2):
    for j in range(2):
        cor_txt = 'white' if cm[i, j] > cm.max() / 2 else '#1a1a1a'
        axes[0].text(j, i, f'{cm[i,j]:,}\n{labels_cm[i][j]}',
                     ha='center', va='center', fontsize=9, color=cor_txt)
axes[0].set(xticks=[0, 1], yticks=[0, 1],
            xticklabels=['Previsto: Não', 'Previsto: Sim'],
            yticklabels=['Real: Não', 'Real: Sim'],
            title='Matriz de Confusão')

# Curva ROC
axes[1].plot(fpr, tpr, color=COR_CHURN, lw=2.5, label=f'Árvore (AUC={auc:.3f})')
axes[1].plot([0, 1], [0, 1], '--', color=COR_NEUTRO, lw=1)
axes[1].fill_between(fpr, tpr, alpha=0.07, color=COR_CHURN)
axes[1].set(xlabel='FPR', ylabel='TPR', title='Curva ROC')
axes[1].legend(fontsize=9)

# Distribuição das probabilidades por classe real
bins = np.linspace(0, 1, 20)
axes[2].hist(y_proba[y_test == 0], bins=bins, color=COR_OK,    alpha=0.65,
             label='Real: Não reduziu', density=True)
axes[2].hist(y_proba[y_test == 1], bins=bins, color=COR_CHURN, alpha=0.65,
             label='Real: Reduziu', density=True)
axes[2].axvline(0.5, color='#1a1a1a', lw=1.3, linestyle='--', label='Threshold 0.5')
axes[2].set(xlabel='Probabilidade Prevista', ylabel='Densidade',
            title='Distribuição das Probabilidades')
axes[2].legend(fontsize=8)

plt.suptitle('Avaliação — Árvore de Decisão', fontweight='bold', fontsize=11)
plt.tight_layout()
plt.show()

print(classification_report(y_test, y_pred, target_names=['Não reduziu', 'Reduziu']))

## 8. Importância das Features

In [ ]:
# ── Importância Gini ──────────────────────────────────────────────
imp_df = pd.DataFrame({
    'feature':    FEATURES,
    'nome':       feature_names_plot,
    'importance': dt.feature_importances_
}).query('importance > 0').sort_values('importance', ascending=True)

fig, axes = plt.subplots(1, 2, figsize=(14, max(4, len(imp_df) * 0.42 + 1)))

# Barras
q75 = imp_df['importance'].quantile(0.75)
cores_imp = [COR_CHURN if v >= q75 else COR_NEUTRO for v in imp_df['importance']]
axes[0].barh(imp_df['nome'], imp_df['importance'], color=cores_imp)
axes[0].set(title='Importância das Features (Gini Impurity)',
            xlabel='Importância relativa')
for i, v in enumerate(imp_df['importance']):
    axes[0].text(v + 0.002, i, f'{v:.3f}', va='center', fontsize=8)

# Acumulada
imp_sorted = imp_df.sort_values('importance', ascending=False)
cum_imp    = imp_sorted['importance'].cumsum()
n_80       = int((cum_imp < 0.80).sum()) + 1

axes[1].bar(range(len(imp_sorted)), imp_sorted['importance'],
            color=[COR_CHURN if i < n_80 else COR_NEUTRO for i in range(len(imp_sorted))])
ax2 = axes[1].twinx()
ax2.plot(range(len(imp_sorted)), cum_imp.values, 'o-', color='#1a1a1a', lw=1.8, ms=5)
ax2.axhline(0.80, color='#b07d1a', lw=1.2, linestyle='--', label='80% acumulado')
ax2.set(ylabel='Importância acumulada', ylim=(0, 1.05))
ax2.legend(fontsize=8)
axes[1].set(
    title=f'{n_80} features explicam 80% do poder preditivo',
    xticks=range(len(imp_sorted)), ylabel='Importância'
)
axes[1].set_xticklabels(
    [n[:14] for n in imp_sorted['nome']], rotation=40, ha='right', fontsize=8)

plt.suptitle('Importância das Features — Árvore de Decisão', fontweight='bold')
plt.tight_layout()
plt.show()

print('Top 5 features por importância Gini:')
for _, r in imp_df.sort_values('importance', ascending=False).head(5).iterrows():
    print(f'  {r["nome"]:<32} {r["importance"]:.4f}')

## 9. Comparação com o Logit

In [ ]:
# ── Logit (com normalização — obrigatório para ele) ────────────────
scaler       = StandardScaler()
Xtr_sc       = scaler.fit_transform(X_train)
Xte_sc       = scaler.transform(X_test)
logit        = LogisticRegression(C=1.0, max_iter=500, random_state=42)
logit.fit(Xtr_sc, y_train)
y_proba_l    = logit.predict_proba(Xte_sc)[:, 1]
y_pred_l     = logit.predict(Xte_sc)

# ── Tabela comparativa ────────────────────────────────────────────
metricas = [
    ('AUC-ROC',   lambda yp, ypr: roc_auc_score(y_test, ypr)),
    ('Acurácia',  lambda yp, ypr: accuracy_score(y_test, yp)),
    ('Recall',    lambda yp, ypr: recall_score(y_test, yp)),
    ('Precisão',  lambda yp, ypr: precision_score(y_test, yp)),
    ('F1-Score',  lambda yp, ypr: f1_score(y_test, yp)),
]

print(f"{'Métrica':<22}  {'Árvore':>10}  {'Logit':>10}  Melhor")
print('─' * 55)
for nome_m, fn in metricas:
    av = fn(y_pred, y_proba)
    lv = fn(y_pred_l, y_proba_l)
    melhor = '✅ Árvore' if av > lv else '✅ Logit'
    print(f'  {nome_m:<20}  {av:>10.3f}  {lv:>10.3f}  {melhor}')

# ── Curvas ROC sobrepostas ─────────────────────────────────────────
fig, ax = plt.subplots(figsize=(7, 5))
for nome_m, ypr, cor in [
    ('Árvore de Decisão',   y_proba,   COR_CHURN),
    ('Regressão Logística', y_proba_l, COR_OK),
]:
    fp, tp, _ = roc_curve(y_test, ypr)
    auc_m = roc_auc_score(y_test, ypr)
    ax.plot(fp, tp, lw=2.5, color=cor, label=f'{nome_m} (AUC={auc_m:.3f})')

ax.plot([0, 1], [0, 1], '--', color=COR_NEUTRO, lw=1)
ax.set(xlabel='FPR', ylabel='TPR', title='Curvas ROC — Comparativo')
ax.legend(fontsize=10)
plt.tight_layout()
plt.show()

print()
print('💡 Quando escolher Árvore vs. Logit?')
print()
print('  Árvore → quando precisar explicar o modelo em reunião com gestores')
print('  Árvore → quando houver interações entre variáveis')
print('  Árvore → quando o time de campo vai aplicar regras manualmente')
print('  Logit  → quando AUC for significativamente maior')
print('  Logit  → quando exigência regulatória pede coeficientes/p-values')
print('  Logit  → quando precisar calibrar threshold via EMP')

## 10. Saída Acionável — Segmentos com Regras Claras

> A grande vantagem operacional da árvore:  
> **cada folha vira um segmento com uma regra que o time aplica sem abrir o Python.**

In [ ]:
# ── Scoring de toda a base ─────────────────────────────────────────
X_full = pd.DataFrame(
    imputer.transform(
        df_model.drop(columns=[TARGET]).reindex(columns=FEATURES, fill_value=0)
    ),
    columns=FEATURES
)

prob_full  = dt.predict_proba(X_full)[:, 1]
folha_full = dt.apply(X_full)   # ID da folha para cada cliente

resultado_full = pd.DataFrame({
    'id_cliente':    df['id_cliente'],
    'segmento_A':    df['segmento_A'],
    'nps_score':     df['nps_score'],
    'canal_digital': df['canal_digital'],
    'prob_churn':    prob_full.round(4),
    'folha_id':      folha_full,
    'alerta':        (prob_full >= 0.5).astype(int),
}).sort_values('prob_churn', ascending=False)

# Perfil de cada folha
resumo = resultado_full.groupby('folha_id').agg(
    n_clientes  = ('id_cliente',    'count'),
    prob_media  = ('prob_churn',    'mean'),
    pct_seg_A   = ('segmento_A',    'mean'),
    nps_medio   = ('nps_score',     'mean'),
    pct_digital = ('canal_digital', 'mean'),
).sort_values('prob_media', ascending=False).round(3)

resumo['risco'] = resumo['prob_media'].apply(
    lambda x: '🔴 ALTO' if x > 0.65 else ('🟡 MÉDIO' if x > 0.35 else '🟢 BAIXO')
)

print('Perfil de cada segmento (folha da árvore):')
print(resumo.to_string())

resultado_full.to_csv('arvore_segmentos_clientes.csv', index=False)
print('\n✅ arvore_segmentos_clientes.csv exportado')

In [ ]:
# ── Visualização executiva ─────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Pizza: distribuição de risco
n_alerta = resultado_full['alerta'].sum()
n_ok     = len(resultado_full) - n_alerta
axes[0].pie(
    [n_ok, n_alerta],
    labels=[f'Baixo/médio risco\n({n_ok:,})', f'Alto risco (alerta)\n({n_alerta:,})'],
    colors=[COR_OK, COR_CHURN],
    autopct='%1.1f%%',
    startangle=90,
    explode=[0, 0.07],
    textprops={'fontsize': 9}
)
axes[0].set_title('Distribuição de Risco na Base', fontweight='bold')

# Heatmap: prob churn por segmento × canal
crosstab = resultado_full.groupby(
    ['segmento_A', 'canal_digital'])['prob_churn'].mean().unstack()
crosstab.index   = ['Seg. B/C', 'Seg. A']
crosstab.columns = ['Tradicional', 'Digital']

im = axes[1].imshow(crosstab.values, cmap='RdYlGn_r', vmin=0, vmax=1)
for i in range(2):
    for j in range(2):
        val = crosstab.values[i, j]
        axes[1].text(j, i, f'{val:.1%}', ha='center', va='center',
                     fontsize=14, fontweight='bold',
                     color='white' if val > 0.55 else '#1a1a1a')
axes[1].set(
    xticks=[0, 1], yticks=[0, 1],
    xticklabels=crosstab.columns,
    yticklabels=crosstab.index,
    title='Probabilidade Média de Churn\nSegmento × Canal'
)
plt.colorbar(im, ax=axes[1], label='Prob. churn', shrink=0.75)

plt.suptitle('Segmentos de Risco — Visão Executiva', fontweight='bold', fontsize=11)
plt.tight_layout()
plt.show()

# Resumo executivo
n_seg_A_alerta = resultado_full[
    (resultado_full['alerta'] == 1) & (resultado_full['segmento_A'] == 1)
].shape[0]
prob_pior = resultado_full[
    (resultado_full['segmento_A'] == 1) & (resultado_full['canal_digital'] == 1)
]['prob_churn'].mean()

print('=== RESUMO EXECUTIVO ===')
print(f'Clientes em alerta:                     {n_alerta:,} ({n_alerta/len(resultado_full):.1%} da base)')
print(f'Destes, do Segmento A:                  {n_seg_A_alerta:,} ({n_seg_A_alerta/n_alerta:.1%})')
print(f'Prob. média — Seg. A + Canal Digital:   {prob_pior:.1%}')
print()
print('Regra principal identificada pela árvore:')
print('  Segmento A + Canal Digital + NPS baixo = grupo de maior risco')
print('  → Ação: campanha de reativação com oferta proporcional ao LTV previsto')

## Síntese: Quando Usar Cada Modelo?

| Situação | Modelo recomendado |
|---|---|
| Explicar para gestores sem Python | **Árvore de Decisão** |
| Time de campo aplica regras manualmente | **Árvore de Decisão** |
| Coeficientes/p-values para auditoria | **Logit** |
| Calibrar threshold via EMP | **Logit** |
| Maximizar AUC sem restrição de interpretabilidade | **XGBoost** |
| Detectar interações complexas | **XGBoost + SHAP** |

---
*A árvore não é melhor nem pior que o Logit — ela responde perguntas diferentes.*  
*Rodar os dois e comparar é sempre uma boa prática.*